In [3]:
import pandas as pd
import re
import numpy as np
from collections import Counter

# Load the Ashok Leyland CSV file
ashok_leyland_data = pd.read_csv('/home/bipin/Documents/kotak/car_model/manufacturer_data/ASHOK LEYLAND.csv')
print(f"Loaded {len(ashok_leyland_data)} records from ASHOK LEYLAND.csv")
ashok_leyland_data.head()

# Function to clean and normalize model names
def clean_model(text):
    if pd.isna(text) or text is None:
        return ""
    # Convert to string
    text = str(text).upper()
    # Remove common prefixes and company names
    text = re.sub(r'ASHOK\s+LEYLAND\s+LTD,?\s*', '', text)
    text = re.sub(r'ASHOK\s+LEYLAND\s+LIMITED,?\s*', '', text)
    text = re.sub(r'ASHOK\s+LEYLAND\s+MOTOR,?\s*', '', text)
    text = re.sub(r'ASHOK\s+LEYLAND,?\s*', '', text)
    text = re.sub(r'ASHOKLEYLAND,?\s*', '', text)
    text = re.sub(r'\"', '', text)
    
    # Remove special characters and extra spaces
    # Fix is here - either place the hyphen at the end or escape it
    text = re.sub(r'[^\w\s/\-]', ' ', text)  # Fixed version - moved hyphen to end
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Create a list of cleaned model names
ashok_leyland_data['clean_model'] = ashok_leyland_data['rc_maker_model'].apply(clean_model)

# Filter out too generic or empty model names
generic_terms = ['LTD', 'BS', 'IV', 'III', 'BS-IV', 'BS-III', 'BSIV', 'BSIII', 'LIMITED', 'MA']
filtered_models = ashok_leyland_data['clean_model'].apply(
    lambda x: x if x and all(term not in x.split() for term in generic_terms) else np.nan
)

# Count model name frequencies
model_counts = Counter([m for models in filtered_models.dropna() for m in models.split()])
model_counts = {k: v for k, v in model_counts.items() if v >= 10}  # Keep only frequent terms

# Print top model keywords by frequency
print("Top model keywords by frequency:")
sorted_counts = sorted(model_counts.items(), key=lambda x: x[1], reverse=True)
for model, count in sorted_counts[:30]:
    print(f"{model}: {count}")

# Define primary Ashok Leyland model keywords
ashokleyland_model_keywords = [
    # Commercial Trucks - Heavy Duty
    "3118", "3718", "3518", "3116", "4018", "4019", "4023", "4218", "4223", "4923", "5525", "6079",
    
    # Commercial Trucks - Medium Duty
    "2518", "2516", "2214", "1616", "1618", "1415", "1214", "1212", "1612", "1613", "1112",
    
    # Light Commercial Vehicles
    "DOST", "BADA", "PARTNER", "BOSS", "ECOMET", "MITR",
    
    # Bus Models
    "VIKING", "CHEETAH", "LYNX", "STAG",
    
    # Common Series/Variants
    "IL", "XL", "TI", "LS", "LE", "LX", "RLS", "SMT", "TT", "HD",
    
    # Specific LCV Models
    "DOST+", "DOST PLUS", "BADA DOST",
    
    # Legacy Models
    "TUSKER", "TAURUS", "COMET",
    
    # Variants for DOST
    "DOST LS", "DOST LE", "DOST LX", "DOST PLUS", "DOST RLS", "DOST RLE", "DOST LX", "DOST LITE",
    
    # Variants for PARTNER
    "PARTNER LS", "PARTNER LE", "PARTNER LLX", "PARTNER SES", "PARTNER XES",
    
    # Variants for ECOMET
    "ECOMET 1012", "ECOMET 1109", "ECOMET 1212", "ECOMET 1214",
    
    # Variants for BOSS
    "BOSS 912", "BOSS 913", "BOSS 1112", "BOSS 1212", "BOSS 1413",
    
    # Tipper Variants
    "TIPPER", "TIP"
]

# Create alias map for common misspellings and variations
ashokleyland_alias_map = {
    # DOST variations
    "DOSTRLE": "DOST RLE",
    "DOST+": "DOST PLUS",
    "DOST REFRESH": "DOST",
    "DOST EXPRESS": "DOST",
    "DOST LS BSIII": "DOST LS",
    "DOST LS BSIV": "DOST LS",
    "DOST LE BSIV": "DOST LE",
    "DOST LX BSIV": "DOST LX",
    "DOST REF": "DOST",
    "DOST BLS": "DOST",
    "DOST+RLS": "DOST PLUS RLS",
    "DOSTRLS": "DOST RLS",
    
    # BADA DOST variations
    "BADADOST": "BADA DOST",
    "BDI3": "BADA DOST I3",
    "BDI4": "BADA DOST I4",
    "BADA DOST I3": "BADA DOST",
    "BADA DOST I4": "BADA DOST",
    "BADA DOST I3+": "BADA DOST",
    
    # PARTNER variations
    "PARTNERLES": "PARTNER LE",
    "PARTNERLLS": "PARTNER LLS",
    "PARTNERLLX": "PARTNER LLX",
    "PARTNERAES": "PARTNER AES",
    "PARTNERSES": "PARTNER SES",
    "PARTNERXES": "PARTNER XES",
    
    # ECOMET variations
    "ECOMETBS": "ECOMET",
    "ECOMATE": "ECOMET",
    "ECOMET1212": "ECOMET 1212",
    "ECOMET1214": "ECOMET 1214",
    "ECOMET1012": "ECOMET 1012",
    "ECOMET1109": "ECOMET 1109",
    
    # BOSS variations
    "BOSS912": "BOSS 912",
    "BOSS913": "BOSS 913",
    "BOSS1112": "BOSS 1112",
    "BOSS1212": "BOSS 1212",
    "BOSS1413": "BOSS 1413",
    
    # Common model number variations
    "N4923": "4923",
    "N3118": "3118",
    "N2518": "2518",
    "U3518": "3518",
    "U2518": "2518",
    "U4019": "4019",
    "TS2214": "2214",
    "CT1616": "1616",
    "CG1613": "1613",
    
    # TUSKER variations
    "TUSKERSUPER": "TUSKER SUPER",
    "TUSKERSUPER2516": "TUSKER SUPER 2516",
    "TUSKERSUPER2214": "TUSKER SUPER 2214",
    "TUSKER SUPER1616": "TUSKER SUPER 1616",
    
    # TAURUS variations
    "TAURUS2516": "TAURUS 2516",
    "TAURUS2513": "TAURUS 2513",
    
    # Bus model variations
    "PSV": "VIKING",
    "ALPSV": "VIKING",
    "ALPSV3": "VIKING",
    "ALPSV4": "VIKING",
    "ALFV": "VIKING",
    "ALFVB": "VIKING",
    
    # Common combinations should map to base models
    "3118IL": "3118",
    "3718IL": "3718",
    "3518IL": "3518",
    "3116IL": "3116",
    "2518IL": "2518",
    "2516IL": "2516",
    "2518T": "2518",
    "1616IL": "1616",
    "1618IL": "1618",
    
    # Generic parts that should be removed
    "BS3": "",
    "BS4": "",
    "BSIII": "",
    "BSIV": "",
    "BSVI": ""
}

# Function to normalize text
def normalize(text):
    # Handle NaN/None values or any non-string types
    if pd.isna(text) or text is None:
        return ""
    # Convert to string if it's not already a string
    if not isinstance(text, str):
        text = str(text)
    # Then proceed with normalization
    return re.sub(r'[^A-Z0-9 ]', '', text.upper())

# Function to extract Ashok Leyland model
def extract_ashokleyland_model(text):
    txt = normalize(text)
    
    # Skip entries that clearly aren't Ashok Leyland vehicles
    non_al = ["TATA", "MAHINDRA", "MARUTI", "HONDA", "HYUNDAI"]
    for brand in non_al:
        if brand in txt:
            return "NOT FOUND"
    
    # First, check for the most specific complete models
    for kw in ["DOST PLUS", "BADA DOST", "ECOMET 1212", "ECOMET 1214", "BOSS 1212", "TUSKER SUPER"]:
        if kw in txt:
            return kw
    
    # Correct aliases
    for bad, good in ashokleyland_alias_map.items():
        if bad in txt:
            txt = txt.replace(bad, good)
    
    # Then search for specific keywords - sort by length to find longer matches first
    for kw in sorted(ashokleyland_model_keywords, key=len, reverse=True):
        if kw in txt:
            # For truck models, check if we can find the exact model number
            if kw in ["3118", "3718", "3518", "2518", "2516", "1616", "1618", "1212", "1214"]:
                # Look for model with variant (like 3118IL)
                model_pattern = re.compile(f"{kw}[A-Z]+")
                matches = model_pattern.findall(txt)
                if matches:
                    return matches[0]
            return kw
            
    return "NOT FOUND"

# Apply extraction to the entire dataset
ashok_leyland_data['ashokleyland_model'] = ashok_leyland_data['rc_maker_model'].apply(extract_ashokleyland_model)

# Display extraction results
model_freq = ashok_leyland_data['ashokleyland_model'].value_counts()
print("\nModel extraction results:")
print(model_freq.head(20))
print(f"\nModels found: {(ashok_leyland_data['ashokleyland_model'] != 'NOT FOUND').sum()} out of {len(ashok_leyland_data)} ({(ashok_leyland_data['ashokleyland_model'] != 'NOT FOUND').sum()/len(ashok_leyland_data)*100:.1f}%)")

# Create a final_model column using fallback logic
ashok_leyland_data['final_model'] = ashok_leyland_data['ashokleyland_model'].apply(lambda x: x if x != "NOT FOUND" else "NOT FOUND")

# Save results to CSV for review
output_path = '/home/bipin/Documents/kotak/car_model/clean_model/ashokleyland_models_extracted.csv'
ashok_leyland_data.to_csv(output_path, index=False)
print(f"\nSaved results to {output_path}")

# Create a frequency mapping of original values to extracted models
mapping_df = ashok_leyland_data.groupby('rc_maker_model')['ashokleyland_model'].agg(
    lambda x: x.value_counts().index[0] if len(x.value_counts()) > 0 else "NOT FOUND"
).reset_index()
mapping_df.columns = ['original_value', 'mapped_model']
mapping_df['count'] = mapping_df['original_value'].map(ashok_leyland_data['rc_maker_model'].value_counts())
mapping_df = mapping_df.sort_values('count', ascending=False)

# Save the mapping for review
mapping_path = '/home/bipin/Documents/kotak/car_model/clean_model/ashokleyland_model_mapping.csv'
mapping_df.to_csv(mapping_path, index=False)
print(f"Saved mapping to {mapping_path}")

# Save the keywords and alias map for reuse
with open('/home/bipin/Documents/kotak/car_model/clean_model/ashokleyland_keywords.py', 'w') as f:
    f.write("ashokleyland_model_keywords = [\n")
    for kw in sorted(ashokleyland_model_keywords):
        f.write(f"    \"{kw}\",\n")
    f.write("]\n\n")
    
    f.write("ashokleyland_alias_map = {\n")
    for bad, good in sorted(ashokleyland_alias_map.items()):
        f.write(f"    \"{bad}\": \"{good}\",\n")
    f.write("}\n")

print("Keywords and alias map saved to ashokleyland_keywords.py")

Loaded 8126 records from ASHOK LEYLAND.csv
Top model keywords by frequency:
AL: 754
H: 473
DOST: 437
IL: 363
WB: 331
ALPSV: 310
CC: 264
A: 239
TUSKER: 230
ECOMET: 226
SUPER: 202
L: 188
T: 185
COMET: 168
2516: 166
1212: 161
1616: 161
LS: 155
BS4: 148
3118: 144
A/L: 141
ASHOK: 135
TIPPER: 123
LEYLAND: 119
BOSS: 118
2518: 118
GOODS: 113
BS3: 111
U: 107
TT: 105

Model extraction results:
ashokleyland_model
NOT FOUND       1434
VIKING           647
3118             324
DOST             312
1616             302
2518             299
LE               230
2516             219
3718             211
3116             202
TUSKER SUPER     184
TIPPER           176
DOST LS          174
COMET            165
2214             162
ECOMET 1212      154
4019             150
LS               130
TUSKER           130
3518             127
Name: count, dtype: int64

Models found: 6692 out of 8126 (82.4%)

Saved results to /home/bipin/Documents/kotak/car_model/clean_model/ashokleyland_models_extracted.csv
Saved 